# Open AI Function Calling -- WHY?

Even with all the good stuff, the OpenAI API was a nightmare for the developers and engineers. Why? They are accustomed to working with structured data types, and working with unstructured data like string is hard.

To get consistent results, developers have to use regular expressions (RegEx) or prompt engineering to extract the information from the text string.

This is where OpenAI's function calling capability comes in. It allows GPT-3.5 and GPT-4 models to take user-defined functions as input and generate structure output. With this, you don't need to write RegEx or perform prompt engineering.

In [ ]:
import openai
import json

In [ ]:
student_1_description = "Susan Johnson is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating."

# A simple prompt to extract information from "student_description" in a JSON format.
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''

response = openai.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': prompt1}]
)
json_object = json.loads(response.choices[0].message.content)
print(json.dumps(json_object, indent=4))


In [ ]:
student_2_description="Ravish Kurpad is a sophomore majoring in computer science at the University of Michigan. He is South Asian Indian American and has a 3.7 GPA. Ravi is an active member of the university's Chess Club and the South Asian Student Association. He hopes to pursue a career in software engineering after graduating."

prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''

response = openai.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': prompt2}]
)
json_object = json.loads(response.choices[0].message.content)
print(json.dumps(json_object, indent=4))

# THE OUTPUT IS NOT CONSISTENT :(
Grades of the first student is “3.8 GPA”, whereas in the second student prompt, we got only the number “3.7”. It is a huge deal when you are building a stable system.
Instead of returning one club, it has returned the list of clubs joined by Ravish. It is also different from the first student.

OpenAI Functions to the Rescue!

In [ ]:
student_custom_functions = [
    {
        'name': 'extract_student_info',
        'description': 'Get the student information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'major': {
                    'type': 'string',
                    'description': 'Major subject.'
                },
                'school': {
                    'type': 'string',
                    'description': 'The university name.'
                },
                'grades': {
                    'type': 'integer',
                    'description': 'GPA of the student.'
                },
                'club': {
                    'type': 'string',
                    'description': 'School club for extracurricular activities. '
                }
                
            }
        }
    }
]

In [ ]:
student_description = [student_1_description,student_2_description]
for sample in student_description:
    response = openai.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': sample}],
        functions = student_custom_functions,
        function_call = 'auto'
    )

    # Loading the response as a JSON object
    json_object = json.loads(response.choices[0].message.function_call.arguments)
    print(json.dumps(json_object, indent=4))

# NICE!
As we can see, we got uniform output. We even got grades in numeric instead of string. Consistent output is essential for creating bug-free AI applications.

# Multiple Custom Functions

You can add multiple custom functions to the chat completion function. In this section, we will see the magical capabilities of OpenAI API and how it automatically selects the correct function and returns the right arguments.

In the Python list of the dictionary, we will add another function called “extract_school_info,” which will help us extract university information from the text.

To achieve this, you have to add another dictionary of a function with name, description, and parameters.

In [ ]:
custom_functions = [
    {
        'name': 'extract_student_info',
        'description': 'Get the student information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'major': {
                    'type': 'string',
                    'description': 'Major subject.'
                },
                'school': {
                    'type': 'string',
                    'description': 'The university name.'
                },
                'grades': {
                    'type': 'integer',
                    'description': 'GPA of the student.'
                },
                'club': {
                    'type': 'string',
                    'description': 'School club for extracurricular activities. '
                }
                
            }
        }
    },
    {
        'name': 'extract_school_info',
        'description': 'Get the school information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the school.'
                },
                'ranking': {
                    'type': 'integer',
                    'description': 'QS world ranking of the school.'
                },
                'country': {
                    'type': 'string',
                    'description': 'Country of the school.'
                },
                'no_of_students': {
                    'type': 'integer',
                    'description': 'Number of students enrolled in the school.'
                }
            }
        }
    }
]

In [ ]:
school_1_description = "Stanford University is a private research university located in Stanford, California, United States. It was founded in 1885 by Leland Stanford and his wife, Jane Stanford, in memory of their only child, Leland Stanford Jr. The university is ra/nked #5 in the world by QS World University Rankings. It has over 17,000 students, including about 7,600 undergraduates and 9,500 graduates23. "

In [ ]:
description = [student_1_description, school_1_description]
for i in description:
    response = openai.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': i}],
        functions = custom_functions,
        function_call = 'auto'
    )

    # Loading the response as a JSON object
    json_object = json.loads(response.choices[0].message.function_call.arguments)
    print(json.dumps(json_object, indent=4))

## NOTE
The GPT-3.5-Turbo model has automatically selected the correct function for different description types. We get perfect JSON output for the student and the school!!

# Application Using Function Calling

In this section, we will build a stable text summarizer that will summarize the school and student information in a certain way.


In [ ]:
def extract_student_info(name, major, school, grades, club):
    
    """Get the student information"""

    return f"{name} is majoring in {major} at {school}. He has {grades} GPA and he is an active member of the university's {club}."

def extract_school_info(name, ranking, country, no_of_students):
    
    """Get the school information"""

    return f"{name} is located in the {country}. The university is ranked #{ranking} in the world with {no_of_students} students."

In [ ]:
# Next, we will create a prompt to extract information from the student description and the school description.

# Create the Python list, which consists of student one description, random prompt, and school one description. The random prompt is added to validate the automatic function calling mechanic.
# We will generate the response using each text in the `descriptions` list.
# If a function call is used, we will get the name of the function and, based on it, apply the relevant arguments to the function using the response. Otherwise, return the normal response.
# Print the outputs of all three samples.

descriptions = [
    student_1_description, 
    "Who was a Abraham Lincoln?",
    school_1_description
                ]
for i, sample in enumerate(descriptions):
    response = openai.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': sample}],
        functions = custom_functions,
        function_call = 'auto'
    )
    
    response_message = response.choices[0].message
    
    if response_message.function_call:
        
        # Which function call was invoked
        function_called = response_message.function_call.name
        
        # Extracting the arguments
        function_args  = json.loads(response_message.function_call.arguments)
        
        # Function names
        available_functions = {
            "extract_school_info": extract_school_info,
            "extract_student_info": extract_student_info
        }
        
        fuction_to_call = available_functions[function_called]
        response_message = fuction_to_call(*list(function_args .values()))
        
    else:
        response_message = response_message.content
    
    print(f"\nSample#{i+1}\n")
    print(response_message)